In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.conformer_small_encoder_config
small_model = conformer.Model(**config)

In [6]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_2:0' shape=(1,) dtype=int32>)

In [7]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.Model(small_model, vocabulary_size = 100, **config)

In [8]:
p = tf.placeholder(tf.int32, [None, None])

In [ ]:
small_logits = small_transducer([v, p], training = True)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

In [ ]:
sess.run(small_logits, feed_dict = {i: y, p: [[1,1,1,1,1]]}).shape